In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

**Часть, отвечающая за тип запроса**

Загрузим обучающие и тестовые данные

In [100]:
data_train = pd.read_csv("/content/drive/My Drive/KDD/KDDTrain+.csv")
data_test = pd.read_csv("/content/drive/My Drive/KDD/KDDTest+.csv")

In [101]:
data_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [5]:
data_test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.00,1.0,1.0,0.04,0.06,0.00,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
1,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,anomaly
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,anomaly


In [103]:
data_test = data_test.drop("service", axis=1)
data_train = data_train.drop("service", axis=1)

X_train/X_test - матрица признаков;

y_train/y_test -  вектор значений;

In [104]:
y_train = data_train['class']
X_train = data_train.drop('class', axis=1)

In [105]:
y_test = data_test['class']
X_test = data_test.drop('class', axis=1)

In [106]:
y_test

0        anomaly
1        anomaly
2         normal
3        anomaly
4        anomaly
          ...   
22539     normal
22540     normal
22541    anomaly
22542     normal
22543    anomaly
Name: class, Length: 22544, dtype: object

**Обработка данных**

Преобразуем вектора значения в булевый тип (normal = 1, anomaly = 0)

In [107]:
y_train = np.where(y_train == "normal", 1, 0)
y_test = np.where(y_test == "normal", 1, 0)

Преобразуем поля protocol_type, service, flag d X_train и X_test в числовой тип

In [108]:
from sklearn import preprocessing #импорт необходимого модуля 

le_protocol = preprocessing.LabelEncoder() #получение меток
le_protocol.fit(X_train["protocol_type"]) #кодирование меток
X_train["protocol_type"] = le_protocol.transform(X_train["protocol_type"]) #преобразование меток
X_test["protocol_type"] = le_protocol.transform(X_test["protocol_type"])

le_flag = preprocessing.LabelEncoder() 
le_flag.fit(X_train["flag"]) 
X_train["flag"] = le_flag.transform(X_train["flag"]) 
X_test["flag"] = le_flag.transform(X_test["flag"])


In [109]:
data_train.groupby("class").agg("count").duration

class
anomaly    58630
normal     67343
Name: duration, dtype: int64

**Модель**

Создадим и обучим модель 

In [11]:
!pip install catboost

     |████████████████████████████████| 65.9MB 75kB/s 


In [12]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from collections import Counter
from tqdm import tqdm
from sklearn.cluster import KMeans

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [114]:
cat = CatBoostClassifier(iterations = 1000, learning_rate = 0.05, depth = 5, loss_function = 'CrossEntropy', eval_metric='Accuracy',
                         random_state = 53)

train_data = Pool(
    data = X_train,
    label = y_train
)

In [115]:
cat.fit(train_data)

0:	learn: 0.9727243	total: 57.7ms	remaining: 57.6s
1:	learn: 0.9667945	total: 114ms	remaining: 57s
2:	learn: 0.9815119	total: 168ms	remaining: 56s
3:	learn: 0.9763044	total: 226ms	remaining: 56.2s
4:	learn: 0.9793765	total: 285ms	remaining: 56.8s
5:	learn: 0.9811229	total: 342ms	remaining: 56.7s
6:	learn: 0.9799243	total: 397ms	remaining: 56.3s
7:	learn: 0.9811309	total: 452ms	remaining: 56.1s
8:	learn: 0.9821390	total: 512ms	remaining: 56.4s
9:	learn: 0.9825836	total: 578ms	remaining: 57.2s
10:	learn: 0.9836870	total: 636ms	remaining: 57.2s
11:	learn: 0.9840204	total: 694ms	remaining: 57.2s
12:	learn: 0.9839886	total: 751ms	remaining: 57s
13:	learn: 0.9847110	total: 807ms	remaining: 56.8s
14:	learn: 0.9848857	total: 864ms	remaining: 56.7s
15:	learn: 0.9849809	total: 916ms	remaining: 56.4s
16:	learn: 0.9857192	total: 981ms	remaining: 56.7s
17:	learn: 0.9860764	total: 1.04s	remaining: 56.7s
18:	learn: 0.9861955	total: 1.1s	remaining: 56.6s
19:	learn: 0.9862351	total: 1.15s	remaining: 56

Посчитаем метрики точности: accuracy, f1, recall, precision

In [116]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
y_pred = cat.predict(X_test)
y_true = y_test
print("Accuracy: ", accuracy_score(y_true, y_pred))
print("Recall: ", recall_score(y_true, y_pred))
print("Precission: ", precision_score(y_true, y_pred))
print("F1: ", f1_score(y_true, y_pred))

Accuracy:  0.7876153300212917
Recall:  0.9715786221810319
Precission:  0.6764895676489567
F1:  0.797616028404768


# **ЗДЕСЬ НАДО ВИЗУАЛИЗИРОВАТЬ СООТНОШЕНИЯ ПРЕДСКАЗАНЫХ МЕТОК**

Конец части, определяющий тип запроса

**Часть, определяющая вид атаки**

Преобразуем предсказанные данные обратно, объединим с тестовой таблицей, а затем оставим только те тестовые данные, где есть аномалия. Это необходимо для дальнейшей классификации атаки.

In [117]:
y_pred = np.where(y_pred == 1, "normal", "anomaly")
X_test["pred_class"] = y_pred
X_test = X_test[X_test["pred_class"] == "anomaly"].drop("pred_class", axis=1)

Загрузим обучающие и тестовые данные для классификации типа атак

In [125]:
train_multi = pd.read_csv("/content/drive/My Drive/KDD/KDDTrain+_meted.csv")
test_multi = pd.read_csv("/content/drive/My Drive/KDD/KDDTest+_meted.csv")

In [119]:
test_multi.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,Unnamed: 42
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.00,1.0,1.0,0.04,0.06,0.00,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [120]:
train_multi.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,Unnamed: 42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


**Подготовка данных**

In [126]:
train_multi = train_multi.drop("Unnamed: 42", axis=1)
test_multi = test_multi.drop("Unnamed: 42", axis=1)
train_multi = train_multi.drop("service", axis=1)
test_multi = test_multi.drop("service", axis=1)

In [127]:
train_multi.shape

(125973, 41)

In [128]:
test_multi.shape

(22543, 41)

Объединим данные из датасета типов атак, т.к. в обучающей тестовой выборке есть непересекающиеся типы. А затем повторно разделим на тестовую и обучающую.

In [129]:
train_multi = pd.merge(train_multi, test_multi, how='outer')

In [130]:
train_multi.shape

(147454, 41)

In [131]:
train_multi = train_multi[train_multi["class"] != "normal"]

In [132]:
train_multi.shape

(70491, 41)

In [133]:
y_multi = train_multi["class"]
X_multi = train_multi.drop("class", axis=1)

In [134]:
le_protocol_multi = preprocessing.LabelEncoder()
le_protocol_multi.fit(X_multi["protocol_type"])
X_multi["protocol_type"] = le_protocol_multi.transform(X_multi["protocol_type"])


le_flag_multi = preprocessing.LabelEncoder()
le_flag_multi.fit(X_multi["flag"])
X_multi["flag"] = le_flag_multi.transform(X_multi["flag"])


In [135]:
le_label_multi = preprocessing.LabelEncoder()
le_label_multi.fit(y_multi)
y_multi = le_label_multi.transform(y_multi)

In [136]:
from sklearn.model_selection import train_test_split
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi, 
                                                                            y_multi, 
                                                                            test_size=0.1, 
                                                                            random_state=42)

In [137]:
cat_multi = CatBoostClassifier(iterations = 1000, learning_rate = 0.1, depth = 5, loss_function = 'MultiClass', eval_metric='Accuracy',
                         random_state = 53)

train_data_multi = Pool(
    data = X_train_multi,
    label = y_train_multi
)


In [138]:
cat_multi.fit(train_data_multi)

0:	learn: 0.8593654	total: 953ms	remaining: 15m 51s
1:	learn: 0.8680664	total: 1.84s	remaining: 15m 18s
2:	learn: 0.8950363	total: 2.79s	remaining: 15m 27s
3:	learn: 0.8990085	total: 3.7s	remaining: 15m 22s
4:	learn: 0.9321574	total: 4.58s	remaining: 15m 11s
5:	learn: 0.9323151	total: 5.45s	remaining: 15m 3s
6:	learn: 0.9389669	total: 6.32s	remaining: 14m 57s
7:	learn: 0.9397393	total: 7.2s	remaining: 14m 52s
8:	learn: 0.9482669	total: 8.07s	remaining: 14m 48s
9:	learn: 0.9502845	total: 8.94s	remaining: 14m 45s
10:	learn: 0.9519238	total: 9.83s	remaining: 14m 43s
11:	learn: 0.9539257	total: 10.7s	remaining: 14m 40s
12:	learn: 0.9572989	total: 11.6s	remaining: 14m 37s
13:	learn: 0.9628631	total: 12.4s	remaining: 14m 36s
14:	learn: 0.9626740	total: 13.3s	remaining: 14m 34s
15:	learn: 0.9629735	total: 14.2s	remaining: 14m 34s
16:	learn: 0.9640926	total: 15.1s	remaining: 14m 33s
17:	learn: 0.9665516	total: 16s	remaining: 14m 32s
18:	learn: 0.9669930	total: 16.9s	remaining: 14m 31s
19:	lear

In [139]:
y_pred_multi = cat_multi.predict(X_test_multi)
y_true_multi = y_test_multi
print("Accuracy: ", accuracy_score(y_pred_multi, y_true_multi))

Accuracy:  0.997872340425532


Теперь преобразуем тестовые данные к тому формату, на котором обучалась мультиклассовая модель.

In [140]:
X_test["protocol_type"] = le_protocol.inverse_transform(X_test["protocol_type"])

X_test["flag"] = le_flag.inverse_transform(X_test["flag"])

In [141]:
X_test.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.0,1.0,1.0,0.04,0.06,0.0,255,10,0.04,0.06,0.0,0.00,0.00,0.00,1.00,1.0
1,0,tcp,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.0,1.0,1.0,0.01,0.06,0.0,255,1,0.00,0.06,0.0,0.00,0.00,0.00,1.00,1.0
3,0,icmp,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.0,0.0,0.0,1.00,0.00,1.0,3,57,1.00,0.00,1.0,0.28,0.00,0.00,0.00,0.0
12,0,tcp,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111,2,0.0,0.0,1.0,1.0,0.02,0.07,0.0,255,2,0.01,0.07,0.0,0.00,0.00,0.00,1.00,1.0
13,0,tcp,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,120,1.0,1.0,0.0,0.0,1.00,0.00,0.0,235,171,0.73,0.07,0.0,0.00,0.69,0.95,0.02,0.0


In [142]:
X_test["protocol_type"] = le_protocol_multi.transform(X_test["protocol_type"])

X_test["flag"] = le_flag_multi.transform(X_test["flag"])

In [143]:
X_test.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.0,1.0,1.0,0.04,0.06,0.0,255,10,0.04,0.06,0.0,0.00,0.00,0.00,1.00,1.0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.0,1.0,1.0,0.01,0.06,0.0,255,1,0.00,0.06,0.0,0.00,0.00,0.00,1.00,1.0
3,0,0,9,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.0,0.0,0.0,1.00,0.00,1.0,3,57,1.00,0.00,1.0,0.28,0.00,0.00,0.00,0.0
12,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111,2,0.0,0.0,1.0,1.0,0.02,0.07,0.0,255,2,0.01,0.07,0.0,0.00,0.00,0.00,1.00,1.0
13,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,120,1.0,1.0,0.0,0.0,1.00,0.00,0.0,235,171,0.73,0.07,0.0,0.00,0.69,0.95,0.02,0.0


In [153]:
y_test_multi_pred = cat_multi.predict(X_test)

Преобразуем обратно метки в названия типов атак

In [154]:
y_test_multi_pred = le_label_multi.inverse_transform(y_test_multi_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [155]:
y_test_multi_pred = y_test_multi_pred.tolist()

In [158]:
from collections import Counter

count_pred = Counter(y_test_multi_pred)

In [159]:
count_pred

Counter({'apache2': 402,
         'back': 353,
         'buffer_overflow': 5,
         'ftp_write': 1,
         'httptunnel': 2,
         'ipsweep': 141,
         'land': 5,
         'loadmodule': 2,
         'mscan': 438,
         'multihop': 1,
         'named': 2,
         'neptune': 4661,
         'nmap': 73,
         'perl': 1,
         'phf': 1,
         'pod': 56,
         'portsweep': 199,
         'processtable': 149,
         'ps': 6,
         'saint': 266,
         'satan': 931,
         'smurf': 680,
         'sqlattack': 2,
         'teardrop': 49,
         'udpstorm': 2,
         'warezmaster': 164,
         'xsnoop': 1,
         'xterm': 4})